# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/david/Dropbox/Udacity/DataEngineering/2 Data Modeling/p2 Data Modeling with Cassandra/p2workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra.

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [23]:
# Make a connection to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute(
    """CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)

#### Set Keyspace

In [7]:
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

## Queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

In this query, we will select columns artist, song, and length. The filters will be on sessionId and itemInSession. To quickly find the data, have it on the same partition, and have it evently distributed, these which will also be used as a composite primary key and as partition key.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song, length FROM music_session WHERE sessionId = 338 AND itemInSession = 4"

In [9]:
## Create the table to answer the query
create_query1 = """CREATE TABLE IF NOT EXISTS music_session
                (sessionId int, itemInSession int, artist text, song text, length double,
                PRIMARY KEY ((sessionId, itemInSession)))
                """

session.execute(create_query1)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_session (sessionId, itemInSession, artist, song, length)
                VALUES (%s, %s, %s, %s, %s)
                """
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Add in the SELECT statement to verify the data was entered into the table
rows = session.execute(query1)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


### Query 2
In this query, we will select columns artist, song, firstName, and LastName. The filters will be on sessionId and itemInSession. The results need to be sorted by itemInSession. To quickly find the data by the filters, have it on the same partition, and have it evently distributed, sessionId and itemInSession will be used as the partition key. To have the items sorted without any additional programming logic, itemInSession is going to be used as a clustering key.

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song, firstName, lastName FROM user_session WHERE userId = 10 AND sessionId = 182"

In [13]:
create_query2 = """CREATE TABLE IF NOT EXISTS user_session
                (userId int, sessionId int, itemInSession int, song text, artist text, firstName text, lastName text,
                PRIMARY KEY ((userId, sessionId), itemInSession))
                """
session.execute(create_query2)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_session (userId, sessionId, itemInSession, song, artist, firstName, lastName)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
rows = session.execute(query2)
for row in rows:
    print(row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')


### Query 3
For this query we need the firstName and the lastName of all users who listened 'All Hands Against His Own'. Since the filter is on the song name, this is going to be the partition key. To ensure uniqueness, we will also add the userId as the composite key. By overwriting duplicates, Cassandra will ensure that we only have one copy of each user and that we have the latest information of such user.

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT firstName, lastName FROM user_song WHERE song = 'All Hands Against His Own'"                    

In [17]:
create_query3 = """CREATE TABLE IF NOT EXISTS user_song
                (song text, userId int, firstName text, lastName text,
                PRIMARY KEY (song, userId))
                """
session.execute(create_query3)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_song (song, userId, firstName, lastName)
                VALUES (%s, %s, %s, %s)
                """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
rows = session.execute(query3)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
drop1 = "DROP TABLE IF EXISTS music_session"
drop2 = "DROP TABLE IF EXISTS user_session"
drop3 = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(drop1)
    session.execute(drop2)
    session.execute(drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()